In [1]:
!pip install datasets huggingface_hub
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|        

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "openai-community/gpt2"  # Replace with your chosen model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
import json

# Load dataset
with open("CDR.json", "r") as f:
    data = json.load(f)

# Define special tokens
PROMPT_TOKEN = "<|prompt|>"
BOT_TOKEN = "<|bot|>"
END_TOKEN = "<|endoftext|>"

# Create formatted dataset
formatted_data = []
for item in data:
    prompt = item["question"]
    response = item["answer"]
    formatted_text = f"{PROMPT_TOKEN} USER: {prompt} {BOT_TOKEN} AGENT: {response} {END_TOKEN}"
    formatted_data.append(formatted_text)

# Save to a new file
with open("formatted_dataset.txt", "w") as f:
    for line in formatted_data:
        f.write(line + "\n")


In [4]:
tokenizer.pad_token = tokenizer.eos_token

In [5]:
from datasets import Dataset

# Convert formatted data into Hugging Face Dataset
formatted_dataset = Dataset.from_dict({"text": formatted_data})



In [12]:
formatted_dataset

Dataset({
    features: ['text'],
    num_rows: 59
})

In [6]:
def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=512
    )
    # Labels are the same as input_ids for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = formatted_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/59 [00:00<?, ? examples/s]

In [7]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 59
})

In [10]:
tokenized_dataset["text"][1]

"<|prompt|> USER: How can I top up my Lyca Mobile number? <|bot|> AGENT: Topping up your Lyca Mobile number in the UK is easy and convenient. Here are the steps to do it:\n\n1. Online: You can top up your Lyca Mobile number online through the official Lyca Mobile website. Here's what you need to do:\n• Visit the Lyca Mobile website.\n• Click on 'Quick Top Up' in Menu or Header of the website.\n• Enter your Lyca Mobile prepay number and the required details.\n• Make the payment, and your prepay number's credit will be topped up.\n\n2. Mobile App: Lyca Mobile also has a mobile app available for both Android and iOS devices. You can download the app from your device's app store and use it to top up your number. Follow the app's instructions for topping up.\n\n3. In-Person: You can visit a Lyca Mobile store or an authorized retailer. They will assist you in topping up your number. Make sure to have your Lyca Mobile number handy.\n\n4. Top-Up Vouchers: You can purchase Lyca Mobile top-up vo

In [8]:
from transformers import Trainer, TrainingArguments

training_args= TrainingArguments(
    output_dir="./results2",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


KeyboardInterrupt: 